In [195]:
import numpy as np
import pandas as pd
import sqlite3

pd.set_option('display.max_columns', None)

## NBA Dataset

For this project, we will be exploring player stats and salary in the NBA. The goal is to use player stats and game outcomes to predict the salary of an NBA player. Using this kind of model, we can figure out who are the most "underpaid" and "overpaid" players in the league. Such a model could also help executives determine what the right amount to pay a player is based on their performance.

### Clean NBA Games DataFrame

In [158]:
games_df = pd.read_csv('../data/raw/csv/games.csv')
games_df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [159]:
games_df.dtypes

GAME_DATE_EST        object
GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

Convert the game date to a Pandas datetime type.

In [160]:
# Convert to pandas datetime
games_df['GAME_DATE_EST'] = pd.to_datetime(games_df['GAME_DATE_EST'])
games_df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


Look for missing values.

In [161]:
games_df.isnull().sum()

GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64

We're really only interested in player data and not game data (other than wins and losses), so we will remove columns with missing data. It also looks like `(HOME_TEAM_ID, TEAM_ID_home)` and `(VISITOR_TEAM_ID, TEAM_ID_away)` are duplicate columns, so TEAM_ID_home and TEAM_ID_away will be dropped.

In [162]:
games_df = games_df[['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'HOME_TEAM_WINS']]
games_df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,0
...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,0
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1


We will check if there are games that were not finished:

In [163]:
games_df['GAME_STATUS_TEXT'].unique()

array(['Final'], dtype=object)

Since all games were final, this column can be dropped.

In [164]:
games_df = games_df.drop(columns=['GAME_STATUS_TEXT'])
games_df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
0,2022-03-12,22101005,1610612748,1610612750,2021,0
1,2022-03-12,22101006,1610612741,1610612739,2021,1
2,2022-03-12,22101007,1610612759,1610612754,2021,0
3,2022-03-12,22101008,1610612744,1610612749,2021,1
4,2022-03-12,22101009,1610612743,1610612761,2021,0
...,...,...,...,...,...,...
25791,2014-10-06,11400007,1610612737,1610612740,2014,1
25792,2014-10-06,11400004,1610612741,1610612764,2014,0
25793,2014-10-06,11400005,1610612747,1610612743,2014,1
25794,2014-10-05,11400002,1610612761,1610612758,2014,1


We also want to use the team names and abbreviation instead of the team ID.

In [165]:
teams_df = pd.read_csv('../data/raw/csv/teams.csv')
teams_df['TEAM_NAME'] = teams_df['CITY'] + ' ' + teams_df['NICKNAME']
teams_df = teams_df[['TEAM_ID', 'ABBREVIATION', 'TEAM_NAME']]
teams_df.head()

,TEAM_ID,ABBREVIATION,TEAM_NAME
0,1610612737,ATL,Atlanta Hawks
1,1610612738,BOS,Boston Celtics
2,1610612740,NOP,New Orleans Pelicans
3,1610612741,CHI,Chicago Bulls
4,1610612742,DAL,Dallas Mavericks


In [166]:
games_df = (games_df.merge(teams_df, left_on='HOME_TEAM_ID', right_on='TEAM_ID')
                    .drop(columns=['HOME_TEAM_ID', 'TEAM_ID']))
games_df = games_df.rename(columns={'ABBREVIATION': 'HOME_ABBREVIATION', 'TEAM_NAME': 'HOME_TEAM_NAME'})

games_df = (games_df.merge(teams_df, left_on='VISITOR_TEAM_ID', right_on='TEAM_ID')
                    .drop(columns=['VISITOR_TEAM_ID', 'TEAM_ID']))
games_df = games_df.rename(columns={'ABBREVIATION': 'VISITOR_ABBREVIATION', 'TEAM_NAME': 'VISITOR_TEAM_NAME'})
games_df

,GAME_DATE_EST,GAME_ID,SEASON,HOME_TEAM_WINS,HOME_ABBREVIATION,HOME_TEAM_NAME,VISITOR_ABBREVIATION,VISITOR_TEAM_NAME
0,2022-03-12,22101005,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
1,2021-05-07,22001005,2020,1,MIA,Miami Heat,MIN,Minnesota Timberwolves
2,2020-02-26,21900868,2019,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
3,2014-04-04,21301135,2013,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
4,2012-12-18,21200359,2012,1,MIA,Miami Heat,MIN,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...
25791,2017-03-29,21601115,2016,0,NYK,New York Knicks,MIA,Miami Heat
25792,2016-02-28,21500883,2015,0,NYK,New York Knicks,MIA,Miami Heat
25793,2015-11-27,21500230,2015,0,NYK,New York Knicks,MIA,Miami Heat
25794,2015-02-20,21400807,2014,0,NYK,New York Knicks,MIA,Miami Heat


Drop duplicate game IDs.

In [167]:
games_df = games_df.drop_duplicates(subset='GAME_ID')
games_df

,GAME_DATE_EST,GAME_ID,SEASON,HOME_TEAM_WINS,HOME_ABBREVIATION,HOME_TEAM_NAME,VISITOR_ABBREVIATION,VISITOR_TEAM_NAME
0,2022-03-12,22101005,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
1,2021-05-07,22001005,2020,1,MIA,Miami Heat,MIN,Minnesota Timberwolves
2,2020-02-26,21900868,2019,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
3,2014-04-04,21301135,2013,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
4,2012-12-18,21200359,2012,1,MIA,Miami Heat,MIN,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...
25791,2017-03-29,21601115,2016,0,NYK,New York Knicks,MIA,Miami Heat
25792,2016-02-28,21500883,2015,0,NYK,New York Knicks,MIA,Miami Heat
25793,2015-11-27,21500230,2015,0,NYK,New York Knicks,MIA,Miami Heat
25794,2015-02-20,21400807,2014,0,NYK,New York Knicks,MIA,Miami Heat


Order it again by the game date.

In [168]:
games_df = games_df.sort_values('GAME_DATE_EST', ascending=False).reset_index(drop=True)
games_df

,GAME_DATE_EST,GAME_ID,SEASON,HOME_TEAM_WINS,HOME_ABBREVIATION,HOME_TEAM_NAME,VISITOR_ABBREVIATION,VISITOR_TEAM_NAME
0,2022-03-12,22101005,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
1,2022-03-12,22101007,2021,0,SAS,San Antonio Spurs,IND,Indiana Pacers
2,2022-03-12,22101010,2021,1,UTA,Utah Jazz,SAC,Sacramento Kings
3,2022-03-12,22101006,2021,1,CHI,Chicago Bulls,CLE,Cleveland Cavaliers
4,2022-03-12,22101008,2021,1,GSW,Golden State Warriors,MIL,Milwaukee Bucks
...,...,...,...,...,...,...,...,...
25762,2003-10-07,10300008,2003,0,PHX,Phoenix Suns,BKN,Brooklyn Nets
25763,2003-10-07,10300004,2003,1,DAL,Dallas Mavericks,ORL,Orlando Magic
25764,2003-10-07,10300009,2003,1,SAC,Sacramento Kings,LAC,Los Angeles Clippers
25765,2003-10-06,10300002,2003,1,MEM,Memphis Grizzlies,MIL,Milwaukee Bucks


### Clean NBA Player Stats DataFrame

In [169]:
stats_df = pd.read_csv('../data/raw/csv/games_details.csv')
stats_df

/var/folders/2w/l1f3dnqs5jg6j1jpdlt6ycvr0000gn/T/ipykernel_97934/140333184.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  stats_df = pd.read_csv('../data/raw/csv/games_details.csv')


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,Anthony,F,NaN,36:22,...,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,Jaden,F,NaN,23:54,...,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0
2,22101005,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,Karl-Anthony,C,NaN,25:17,...,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0
3,22101005,1610612750,MIN,Minnesota,1627736,Malik Beasley,Malik,G,NaN,30:52,...,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0
4,22101005,1610612750,MIN,Minnesota,1626156,D'Angelo Russell,D'Angelo,G,NaN,33:46,...,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645948,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,19,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
645949,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,23,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
645950,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,15,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
645951,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,19,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


Check for missing values.

In [170]:
stats_df.isnull().sum()

GAME_ID                   0
TEAM_ID                   0
TEAM_ABBREVIATION         0
TEAM_CITY                 0
PLAYER_ID                 0
PLAYER_NAME               0
NICKNAME             615591
START_POSITION       398738
COMMENT              540351
MIN                  105603
FGM                  105603
FGA                  105603
FG_PCT               105603
FG3M                 105603
FG3A                 105603
FG3_PCT              105603
FTM                  105603
FTA                  105603
FT_PCT               105603
OREB                 105603
DREB                 105603
REB                  105603
AST                  105603
STL                  105603
BLK                  105603
TO                   105603
PF                   105603
PTS                  105603
PLUS_MINUS           129264
dtype: int64

The player nickname and comment field is not needed, so we can take that out.

In [171]:
stats_df = stats_df.drop(columns=['NICKNAME', 'COMMENT'])

It looks like 105603 rows are missing values for all the major stats — to check, we get these specific rows:

In [172]:
stat_categories = stats_df.columns[7:]
stat_categories

Index(['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [173]:
null_rows = stats_df[stats_df[stat_categories].isnull().all(axis=1)]
null_rows

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
10,22101005,1610612750,MIN,Minnesota,1630195,Leandro Bolmaro,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,22101005,1610612750,MIN,Minnesota,1630233,Nathan Knight,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,22101005,1610612750,MIN,Minnesota,1627774,Jake Layman,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,22101005,1610612748,MIA,Miami,2617,Udonis Haslem,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,22101005,1610612748,MIA,Miami,1630209,Omer Yurtseven,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643271,21200003,1610612747,LAL,Los Angeles,203135,Robert Sacre,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643283,21200001,1610612764,WAS,Washington,201858,Cartier Martin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643294,21200001,1610612739,CLE,Cleveland,201956,Omri Casspi,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643295,21200001,1610612739,CLE,Cleveland,202720,Jon Leuer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Delete these rows from the dataframe.

In [174]:
stats_df = stats_df.drop(null_rows.index).reset_index(drop=True)
stats_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,F,36:22,4.0,10.0,...,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,F,23:54,6.0,8.0,...,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0
2,22101005,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,...,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0
3,22101005,1610612750,MIN,Minnesota,1627736,Malik Beasley,G,30:52,4.0,9.0,...,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0
4,22101005,1610612750,MIN,Minnesota,1626156,D'Angelo Russell,G,33:46,3.0,13.0,...,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540345,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,19,4.0,9.0,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
540346,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,23,7.0,11.0,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
540347,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,15,3.0,7.0,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
540348,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,19,1.0,1.0,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


Check these rows one more time.

In [175]:
stats_df.isnull().sum()

GAME_ID                   0
TEAM_ID                   0
TEAM_ABBREVIATION         0
TEAM_CITY                 0
PLAYER_ID                 0
PLAYER_NAME               0
START_POSITION       293135
MIN                       0
FGM                       0
FGA                       0
FG_PCT                    0
FG3M                      0
FG3A                      0
FG3_PCT                   0
FTM                       0
FTA                       0
FT_PCT                    0
OREB                      0
DREB                      0
REB                       0
AST                       0
STL                       0
BLK                       0
TO                        0
PF                        0
PTS                       0
PLUS_MINUS            23661
dtype: int64

For `START_POSITION`, the null values are for non-starting players for every game, so that is valid. We check the null rows for `PLUS_MINUS`:

In [176]:
null_rows = stats_df[stats_df['PLUS_MINUS'].isnull()]
null_rows

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
94912,10800114,1610612759,SAS,San Antonio,1477,Bruce Bowen,NaN,19,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,NaN
94913,10800114,1610612759,SAS,San Antonio,2824,Desmon Farmer,NaN,20,1.0,4.0,...,0.0,3.0,3.0,2.0,0.0,0.0,1.0,3.0,2.0,NaN
94914,10800114,1610612759,SAS,San Antonio,101177,Fabricio Oberto,NaN,23,4.0,5.0,...,1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,9.0,NaN
94915,10800114,1610612759,SAS,San Antonio,2137,Ime Udoka,NaN,21,5.0,8.0,...,2.0,5.0,7.0,1.0,2.0,0.0,0.0,3.0,11.0,NaN
94916,10800114,1610612759,SAS,San Antonio,1521,Jacque Vaughn,NaN,16,0.0,2.0,...,1.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540345,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,19,4.0,9.0,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
540346,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,23,7.0,11.0,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
540347,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,15,3.0,7.0,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
540348,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,19,1.0,1.0,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


It looks like for these games, +/- was not tracked. Since they only make up a small portion of the data, we will drop these rows as well.

In [177]:
stats_df = stats_df.drop(null_rows.index).reset_index(drop=True)
stats_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,F,36:22,4.0,10.0,...,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,F,23:54,6.0,8.0,...,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0
2,22101005,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,...,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0
3,22101005,1610612750,MIN,Minnesota,1627736,Malik Beasley,G,30:52,4.0,9.0,...,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0
4,22101005,1610612750,MIN,Minnesota,1626156,D'Angelo Russell,G,33:46,3.0,13.0,...,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516684,21200001,1610612739,CLE,Cleveland,101139,CJ Miles,NaN,17:42,1.0,5.0,...,0.0,4.0,4.0,1.0,0.0,0.0,3.0,0.0,2.0,2.0
516685,21200001,1610612739,CLE,Cleveland,203092,Tyler Zeller,NaN,14:53,2.0,4.0,...,0.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,5.0,4.0
516686,21200001,1610612739,CLE,Cleveland,200789,Daniel Gibson,NaN,16:11,3.0,5.0,...,1.0,2.0,3.0,1.0,0.0,1.0,0.0,2.0,10.0,-9.0
516687,21200001,1610612739,CLE,Cleveland,2575,Luke Walton,NaN,12:14,1.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,-11.0


Adjust to use the team abbreviation and name instead of the ID.

In [178]:
stats_df = stats_df.merge(teams_df, on='TEAM_ID')
stats_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,ABBREVIATION,TEAM_NAME
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,F,36:22,4.0,10.0,...,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,F,23:54,6.0,8.0,...,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0,MIN,Minnesota Timberwolves
2,22101005,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,...,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0,MIN,Minnesota Timberwolves
3,22101005,1610612750,MIN,Minnesota,1627736,Malik Beasley,G,30:52,4.0,9.0,...,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0,MIN,Minnesota Timberwolves
4,22101005,1610612750,MIN,Minnesota,1626156,D'Angelo Russell,G,33:46,3.0,13.0,...,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0,MIN,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516684,21200014,1610612760,OKC,Oklahoma City,2555,Nick Collison,NaN,20:01,2.0,6.0,...,1.0,0.0,1.0,1.0,2.0,2.0,6.0,4.0,OKC,Oklahoma City Thunder
516685,21200014,1610612760,OKC,Oklahoma City,2755,Kevin Martin,NaN,32:31,4.0,10.0,...,2.0,5.0,1.0,0.0,1.0,2.0,15.0,6.0,OKC,Oklahoma City Thunder
516686,21200014,1610612760,OKC,Oklahoma City,201934,Hasheem Thabeet,NaN,12:09,0.0,0.0,...,2.0,0.0,2.0,1.0,1.0,2.0,1.0,-2.0,OKC,Oklahoma City Thunder
516687,21200014,1610612760,OKC,Oklahoma City,201953,Eric Maynor,NaN,10:33,3.0,6.0,...,3.0,0.0,0.0,0.0,1.0,0.0,7.0,-4.0,OKC,Oklahoma City Thunder


See where the team abbreviations do not match:

In [179]:
stats_df[stats_df['TEAM_ABBREVIATION'] != stats_df['ABBREVIATION']]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,...,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,ABBREVIATION,TEAM_NAME
416689,40800155,1610612740,NOH,New Orleans,978,Peja Stojakovic,F,41:28,5.0,12.0,...,4.0,1.0,1.0,1.0,1.0,3.0,12.0,-28.0,NOP,New Orleans Pelicans
416690,40800155,1610612740,NOH,New Orleans,2561,David West,F,34:31,8.0,16.0,...,9.0,2.0,2.0,0.0,2.0,4.0,24.0,-9.0,NOP,New Orleans Pelicans
416691,40800155,1610612740,NOH,New Orleans,200756,Hilton Armstrong,C,18:28,2.0,3.0,...,6.0,1.0,0.0,1.0,1.0,6.0,7.0,-1.0,NOP,New Orleans Pelicans
416692,40800155,1610612740,NOH,New Orleans,2446,Rasual Butler,G,31:07,3.0,11.0,...,2.0,1.0,0.0,1.0,3.0,2.0,7.0,-15.0,NOP,New Orleans Pelicans
416693,40800155,1610612740,NOH,New Orleans,101108,Chris Paul,G,46:23,5.0,16.0,...,6.0,10.0,3.0,0.0,3.0,3.0,12.0,-24.0,NOP,New Orleans Pelicans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506563,20300015,1610612760,SEA,Seattle,1738,Ansu Sesay,NaN,17:20,3.0,10.0,...,4.0,0.0,0.0,2.0,1.0,2.0,6.0,-7.0,OKC,Oklahoma City Thunder
506564,20300015,1610612760,SEA,Seattle,2501,Reggie Evans,NaN,25:07,2.0,4.0,...,4.0,0.0,0.0,0.0,1.0,2.0,6.0,-1.0,OKC,Oklahoma City Thunder
506565,20300015,1610612760,SEA,Seattle,2557,Luke Ridnour,NaN,18:55,2.0,9.0,...,2.0,3.0,1.0,0.0,1.0,0.0,8.0,-3.0,OKC,Oklahoma City Thunder
506566,20300015,1610612760,SEA,Seattle,2499,Richie Frahm,NaN,6:57,0.0,3.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OKC,Oklahoma City Thunder


In this case, there are teams which changed their name (e.g., New Orleans Hornets -> New Orleans Pelicans). Since we are using the new names and abbreviations for the games dataframe (`games_df`), we should use the same names and abbreviations for the stats dataframe.

In [180]:
stats_df = stats_df.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY'])
stats_df = stats_df.rename(columns={'ABBREVIATION': 'TEAM_ABBREVIATION'})
stats_df

,GAME_ID,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME
0,22101005,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,...,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves
1,22101005,1630183,Jaden McDaniels,F,23:54,6.0,8.0,0.750,1.0,3.0,...,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0,MIN,Minnesota Timberwolves
2,22101005,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,0.444,1.0,3.0,...,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0,MIN,Minnesota Timberwolves
3,22101005,1627736,Malik Beasley,G,30:52,4.0,9.0,0.444,4.0,9.0,...,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0,MIN,Minnesota Timberwolves
4,22101005,1626156,D'Angelo Russell,G,33:46,3.0,13.0,0.231,1.0,6.0,...,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0,MIN,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516684,21200014,2555,Nick Collison,NaN,20:01,2.0,6.0,0.333,0.0,0.0,...,1.0,0.0,1.0,1.0,2.0,2.0,6.0,4.0,OKC,Oklahoma City Thunder
516685,21200014,2755,Kevin Martin,NaN,32:31,4.0,10.0,0.400,3.0,6.0,...,2.0,5.0,1.0,0.0,1.0,2.0,15.0,6.0,OKC,Oklahoma City Thunder
516686,21200014,201934,Hasheem Thabeet,NaN,12:09,0.0,0.0,0.000,0.0,0.0,...,2.0,0.0,2.0,1.0,1.0,2.0,1.0,-2.0,OKC,Oklahoma City Thunder
516687,21200014,201953,Eric Maynor,NaN,10:33,3.0,6.0,0.500,1.0,2.0,...,3.0,0.0,0.0,0.0,1.0,0.0,7.0,-4.0,OKC,Oklahoma City Thunder


### Clean NBA Player Salary DataFrame

In [181]:
salary_df = pd.read_csv('../data/raw/csv/salaries.csv')
salary_df

,playerName,seasonStartYear,salary,inflationAdjSalary
0,Michael Jordan,1996,"$30,140,000","$52,258,566"
1,Horace Grant,1996,"$14,857,000","$25,759,971"
2,Reggie Miller,1996,"$11,250,000","$19,505,934"
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585"
4,Gary Payton,1996,"$10,212,000","$17,706,187"
...,...,...,...,...
11578,Paul Watson,2019,"$79,568","$84,399"
11579,Jarrell Brantley,2019,"$79,568","$84,399"
11580,Justin Wright-Foreman,2019,"$79,568","$84,399"
11581,Garrison Mathews,2019,"$79,568","$84,399"


We want to convert the salary columns into numbers:

In [182]:
salary_df = salary_df.rename(columns={'playerName': 'PLAYER_NAME', 'seasonStartYear': 'SEASON_START',
                                    'salary': 'SALARY', 'inflationAdjSalary': 'INFLATION_ADJ_SALARY'})

salary_df['SALARY'] = salary_df['SALARY'].str.replace(',', '', regex=False)
salary_df['INFLATION_ADJ_SALARY'] = salary_df['INFLATION_ADJ_SALARY'].str.replace(',', '', regex=False)
salary_df['SALARY'] = salary_df['SALARY'].str.replace('$', '', regex=False)
salary_df['INFLATION_ADJ_SALARY'] = salary_df['INFLATION_ADJ_SALARY'].str.replace('$', '', regex=False)
salary_df['SALARY'] = pd.to_numeric(salary_df['SALARY'])
salary_df['INFLATION_ADJ_SALARY'] = pd.to_numeric(salary_df['INFLATION_ADJ_SALARY'])

salary_df

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
0,Michael Jordan,1996,30140000,52258566
1,Horace Grant,1996,14857000,25759971
2,Reggie Miller,1996,11250000,19505934
3,Shaquille O'Neal,1996,10714000,18576585
4,Gary Payton,1996,10212000,17706187
...,...,...,...,...
11578,Paul Watson,2019,79568,84399
11579,Jarrell Brantley,2019,79568,84399
11580,Justin Wright-Foreman,2019,79568,84399
11581,Garrison Mathews,2019,79568,84399


Remove data prior to 2003 (earliest season in `games_df`):

In [183]:
salary_df = salary_df[salary_df['SEASON_START'] >= 2003]
salary_df

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
3157,Kevin Garnett,2003,28000000,41412563
3158,Shaquille O'Neal,2003,24749999,36605747
3159,Dikembe Mutombo,2003,17714291,26199792
3160,Rasheed Wallace,2003,17000000,25143342
3161,Allan Houston,2003,15937500,23571883
...,...,...,...,...
11578,Paul Watson,2019,79568,84399
11579,Jarrell Brantley,2019,79568,84399
11580,Justin Wright-Foreman,2019,79568,84399
11581,Garrison Mathews,2019,79568,84399


This dataframe only seems to have data up to the 2019-2020 season:

In [184]:
salary_df['SEASON_START'].max()

2019

We augment the dataset with salary data from 2020-2021 + 2021-2022 seasons.

In [185]:
con = sqlite3.connect('../data/raw/basketball.sqlite')
current_salary_df = pd.read_sql_query('SELECT * FROM Player_Salary', con)
current_salary_df

,slugSeason,nameTeam,namePlayer,statusPlayer,isFinalSeason,isWaived,isOnRoster,isNonGuaranteed,isTeamOption,isPlayerOption,typeContractDetail,value
0,2020-21,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
1,2021-22,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
2,2022-23,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
3,2023-24,Atlanta Hawks,Bogdan Bogdanovic,current roster,1,0,1,0,0,1,Player Option,18000000.0
4,2020-21,Atlanta Hawks,Brandon Goodwin,current roster,0,0,1,0,0,0,Guaranteed,1701593.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1287,2020-21,Washington Wizards,Russell Westbrook,current roster,0,0,1,0,0,0,Guaranteed,41358814.0
1288,2021-22,Washington Wizards,Russell Westbrook,current roster,0,0,1,0,0,0,Guaranteed,44211146.0
1289,2022-23,Washington Wizards,Russell Westbrook,current roster,1,0,1,0,0,1,Player Option,47063478.0
1290,2020-21,Washington Wizards,Thomas Bryant,current roster,0,0,1,0,0,0,Guaranteed,8333333.0


We only want current players and their current deals:

In [186]:
current_salary_df['SEASON_START'] = current_salary_df['slugSeason'].str.slice(stop=4)
current_salary_df = current_salary_df[current_salary_df['statusPlayer'] == 'current roster']
current_salary_df = current_salary_df[current_salary_df['SEASON_START'].isin(['2020', '2021'])]
current_salary_df

,slugSeason,nameTeam,namePlayer,statusPlayer,isFinalSeason,isWaived,isOnRoster,isNonGuaranteed,isTeamOption,isPlayerOption,typeContractDetail,value,SEASON_START
0,2020-21,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0,2020
1,2021-22,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0,2021
4,2020-21,Atlanta Hawks,Brandon Goodwin,current roster,0,0,1,0,0,0,Guaranteed,1701593.0,2020
5,2021-22,Atlanta Hawks,Brandon Goodwin,current roster,1,0,1,0,0,0,Qualifying Offer,2126991.0,2021
6,2020-21,Atlanta Hawks,Bruno Fernando,current roster,0,0,1,0,0,0,Guaranteed,1517981.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,2021-22,Washington Wizards,Rui Hachimura,current roster,0,0,1,0,0,0,Guaranteed,4916160.0,2021
1287,2020-21,Washington Wizards,Russell Westbrook,current roster,0,0,1,0,0,0,Guaranteed,41358814.0,2020
1288,2021-22,Washington Wizards,Russell Westbrook,current roster,0,0,1,0,0,0,Guaranteed,44211146.0,2021
1290,2020-21,Washington Wizards,Thomas Bryant,current roster,0,0,1,0,0,0,Guaranteed,8333333.0,2020


We also adjust the value of the 2020-2021 season for inflation. The rate was 4.7%, so we multiply by 1.047.

In [187]:
def adj_inflation(row):
    if row['SEASON_START'] == '2020':
        return int(row['value'] * 1.047)
    return int(row['value'])


current_salary_df['INFLATION_ADJ_SALARY'] = current_salary_df.apply(adj_inflation, axis=1)

We now extract only the relevant columns:

In [188]:
current_salary_df = current_salary_df[['namePlayer', 'SEASON_START', 'value', 'INFLATION_ADJ_SALARY']]
current_salary_df = current_salary_df.rename(columns={'namePlayer': 'PLAYER_NAME', 'value': 'SALARY'})
current_salary_df['SEASON_START'] = pd.to_numeric(current_salary_df['SEASON_START'])
current_salary_df

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
0,Bogdan Bogdanovic,2020,18000000.0,18846000
1,Bogdan Bogdanovic,2021,18000000.0,18000000
4,Brandon Goodwin,2020,1701593.0,1781567
5,Brandon Goodwin,2021,2126991.0,2126991
6,Bruno Fernando,2020,1517981.0,1589326
...,...,...,...,...
1284,Rui Hachimura,2021,4916160.0,4916160
1287,Russell Westbrook,2020,41358814.0,43302678
1288,Russell Westbrook,2021,44211146.0,44211146
1290,Thomas Bryant,2020,8333333.0,8724999


Join it with `salary_df` to create a large salary dataframe.

In [189]:
salary_df = pd.concat([salary_df, current_salary_df])
salary_df

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
3157,Kevin Garnett,2003,28000000.0,41412563
3158,Shaquille O'Neal,2003,24749999.0,36605747
3159,Dikembe Mutombo,2003,17714291.0,26199792
3160,Rasheed Wallace,2003,17000000.0,25143342
3161,Allan Houston,2003,15937500.0,23571883
...,...,...,...,...
1284,Rui Hachimura,2021,4916160.0,4916160
1287,Russell Westbrook,2020,41358814.0,43302678
1288,Russell Westbrook,2021,44211146.0,44211146
1290,Thomas Bryant,2020,8333333.0,8724999


We remove any duplicates that exist:

In [190]:
salary_df[salary_df.duplicated(subset=['PLAYER_NAME', 'SEASON_START'])]

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
8346,Tony Mitchell,2013,28834.0,33550


In [191]:
salary_df = salary_df.drop_duplicates(subset=['PLAYER_NAME', 'SEASON_START'])
salary_df

,PLAYER_NAME,SEASON_START,SALARY,INFLATION_ADJ_SALARY
3157,Kevin Garnett,2003,28000000.0,41412563
3158,Shaquille O'Neal,2003,24749999.0,36605747
3159,Dikembe Mutombo,2003,17714291.0,26199792
3160,Rasheed Wallace,2003,17000000.0,25143342
3161,Allan Houston,2003,15937500.0,23571883
...,...,...,...,...
1284,Rui Hachimura,2021,4916160.0,4916160
1287,Russell Westbrook,2020,41358814.0,43302678
1288,Russell Westbrook,2021,44211146.0,44211146
1290,Thomas Bryant,2020,8333333.0,8724999


### Ensure that the three tables (`games_df`, `stats_df`, `salary_df`) can be merged:

Merge `stats_df` and `games_df`.

In [192]:
stats_df.shape

(516689, 26)

In [193]:
games_df.shape

(25767, 8)

In [196]:
df = stats_df.merge(games_df, on='GAME_ID')
df

,GAME_ID,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_EST,SEASON,HOME_TEAM_WINS,HOME_ABBREVIATION,HOME_TEAM_NAME,VISITOR_ABBREVIATION,VISITOR_TEAM_NAME
0,22101005,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.00,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves,2022-03-12,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
1,22101005,1630183,Jaden McDaniels,F,23:54,6.0,8.0,0.750,1.0,3.0,0.333,1.0,1.0,1.00,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0,MIN,Minnesota Timberwolves,2022-03-12,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
2,22101005,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,0.444,1.0,3.0,0.333,6.0,8.0,0.75,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0,MIN,Minnesota Timberwolves,2022-03-12,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
3,22101005,1627736,Malik Beasley,G,30:52,4.0,9.0,0.444,4.0,9.0,0.444,0.0,0.0,0.00,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0,MIN,Minnesota Timberwolves,2022-03-12,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
4,22101005,1626156,D'Angelo Russell,G,33:46,3.0,13.0,0.231,1.0,6.0,0.167,7.0,7.0,1.00,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0,MIN,Minnesota Timberwolves,2022-03-12,2021,0,MIA,Miami Heat,MIN,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516684,21200183,201566,Russell Westbrook,G,41:51,11.0,23.0,0.478,2.0,5.0,0.400,6.0,6.0,1.00,1.0,4.0,5.0,9.0,2.0,1.0,3.0,2.0,30.0,8.0,OKC,Oklahoma City Thunder,2012-11-24,2012,0,PHI,Philadelphia 76ers,OKC,Oklahoma City Thunder
516685,21200183,2755,Kevin Martin,NaN,37:23,2.0,9.0,0.222,1.0,5.0,0.200,1.0,1.0,1.00,0.0,4.0,4.0,0.0,2.0,0.0,1.0,2.0,6.0,11.0,OKC,Oklahoma City Thunder,2012-11-24,2012,0,PHI,Philadelphia 76ers,OKC,Oklahoma City Thunder
516686,21200183,2555,Nick Collison,NaN,21:17,5.0,6.0,0.833,0.0,0.0,0.000,4.0,4.0,1.00,3.0,2.0,5.0,0.0,0.0,0.0,1.0,2.0,14.0,5.0,OKC,Oklahoma City Thunder,2012-11-24,2012,0,PHI,Philadelphia 76ers,OKC,Oklahoma City Thunder
516687,21200183,201934,Hasheem Thabeet,NaN,8:30,0.0,1.0,0.000,0.0,0.0,0.000,2.0,2.0,1.00,1.0,2.0,3.0,0.0,1.0,0.0,0.0,2.0,2.0,5.0,OKC,Oklahoma City Thunder,2012-11-24,2012,0,PHI,Philadelphia 76ers,OKC,Oklahoma City Thunder


We reduce the game data to return a binary (0,1) variable on whether the person was on the winning team that night.

In [197]:
def won_game(row):
    if row['TEAM_ABBREVIATION'] == row['HOME_ABBREVIATION']:
        return row['HOME_TEAM_WINS']
    elif row['TEAM_ABBREVIATION'] == row['VISITOR_ABBREVIATION']:
        return 1 - row['HOME_TEAM_WINS']

df['WON_GAME'] = df.apply(won_game, axis=1)
df = df.drop(columns=['GAME_ID', 'HOME_ABBREVIATION', 'HOME_TEAM_WINS', 'HOME_ABBREVIATION',
                      'HOME_TEAM_NAME', 'VISITOR_ABBREVIATION', 'VISITOR_TEAM_NAME'])
df

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_EST,SEASON,WON_GAME
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.00,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1
1,1630183,Jaden McDaniels,F,23:54,6.0,8.0,0.750,1.0,3.0,0.333,1.0,1.0,1.00,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1
2,1626157,Karl-Anthony Towns,C,25:17,4.0,9.0,0.444,1.0,3.0,0.333,6.0,8.0,0.75,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1
3,1627736,Malik Beasley,G,30:52,4.0,9.0,0.444,4.0,9.0,0.444,0.0,0.0,0.00,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1
4,1626156,D'Angelo Russell,G,33:46,3.0,13.0,0.231,1.0,6.0,0.167,7.0,7.0,1.00,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516684,201566,Russell Westbrook,G,41:51,11.0,23.0,0.478,2.0,5.0,0.400,6.0,6.0,1.00,1.0,4.0,5.0,9.0,2.0,1.0,3.0,2.0,30.0,8.0,OKC,Oklahoma City Thunder,2012-11-24,2012,1
516685,2755,Kevin Martin,NaN,37:23,2.0,9.0,0.222,1.0,5.0,0.200,1.0,1.0,1.00,0.0,4.0,4.0,0.0,2.0,0.0,1.0,2.0,6.0,11.0,OKC,Oklahoma City Thunder,2012-11-24,2012,1
516686,2555,Nick Collison,NaN,21:17,5.0,6.0,0.833,0.0,0.0,0.000,4.0,4.0,1.00,3.0,2.0,5.0,0.0,0.0,0.0,1.0,2.0,14.0,5.0,OKC,Oklahoma City Thunder,2012-11-24,2012,1
516687,201934,Hasheem Thabeet,NaN,8:30,0.0,1.0,0.000,0.0,0.0,0.000,2.0,2.0,1.00,1.0,2.0,3.0,0.0,1.0,0.0,0.0,2.0,2.0,5.0,OKC,Oklahoma City Thunder,2012-11-24,2012,1


Now merge `df` and `salary_df`.

In [198]:
df = df.rename(columns={'SEASON': 'SEASON_START'})
df.merge(salary_df, on=['PLAYER_NAME', 'SEASON_START'])

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_EST,SEASON_START,WON_GAME,SALARY,INFLATION_ADJ_SALARY
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.000,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1,10245480.0,10245480
1,1630162,Anthony Edwards,F,34:27,9.0,19.0,0.474,4.0,11.0,0.364,3.0,3.0,1.000,0.0,3.0,3.0,5.0,1.0,0.0,0.0,3.0,25.0,-3.0,MIN,Minnesota Timberwolves,2022-03-11,2021,0,10245480.0,10245480
2,1630162,Anthony Edwards,F,25:29,7.0,15.0,0.467,2.0,8.0,0.250,0.0,0.0,0.000,0.0,1.0,1.0,3.0,2.0,1.0,4.0,2.0,16.0,7.0,MIN,Minnesota Timberwolves,2022-03-09,2021,1,10245480.0,10245480
3,1630162,Anthony Edwards,F,32:22,7.0,13.0,0.538,1.0,5.0,0.200,2.0,2.0,1.000,2.0,2.0,4.0,4.0,4.0,0.0,4.0,5.0,17.0,7.0,MIN,Minnesota Timberwolves,2022-02-28,2021,1,10245480.0,10245480
4,1630162,Anthony Edwards,F,37:46,5.0,13.0,0.385,1.0,6.0,0.167,4.0,6.0,0.667,1.0,2.0,3.0,5.0,1.0,1.0,2.0,3.0,15.0,-21.0,MIN,Minnesota Timberwolves,2022-02-25,2021,0,10245480.0,10245480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473751,1628473,Yakuba Ouattara,NaN,3:09,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,-7.0,BKN,Brooklyn Nets,2017-10-03,2017,1,77250.0,85683
473752,201582,Alexis Ajinca,NaN,8:30,0.0,1.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,-10.0,NOP,New Orleans Pelicans,2017-10-13,2017,0,4961798.0,5503462
473753,201582,Alexis Ajinca,NaN,11:49,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,0.000,0.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,-1.0,NOP,New Orleans Pelicans,2017-10-06,2017,0,4961798.0,5503462
473754,1627818,Kaleb Tarczewski,NaN,20:55,2.0,3.0,0.667,0.0,0.0,0.000,1.0,2.0,0.500,1.0,9.0,10.0,0.0,0.0,2.0,2.0,3.0,5.0,-12.0,OKC,Oklahoma City Thunder,2016-10-13,2016,0,75000.0,84546


There are missing rows: some of the names don't align. We try fuzzy matching to get as many matches as possible.

In [199]:
# number of names that are not in the salary dataframe
unusued_names = df[~df['PLAYER_NAME'].isin(salary_df['PLAYER_NAME'])]['PLAYER_NAME'].unique()
unusued_names_2 = salary_df[~salary_df['PLAYER_NAME'].isin(df['PLAYER_NAME'])]['PLAYER_NAME'].unique()

In [200]:
import difflib

matching_names = []
for name in unusued_names:
    close_matches = difflib.get_close_matches(name, unusued_names_2, cutoff=0.75)
    if len(close_matches) > 0:
        matching_names.append([name, close_matches[0]])

matching_names

[['P.J. Tucker', 'PJ Tucker'],
 ['Wendell Carter Jr.', 'Wendell Carter Jr'],
 ['R.J. Hampton', 'RJ Hampton'],
 ['Aleksej Pokusevski', 'Aleksej Pokuesvski'],
 ['Brandon Williams', 'Brandon Wallace'],
 ['Otto Porter Jr.', 'Otto Porter'],
 ['Paul Reed', 'Paul Reed Jr'],
 ['Ziaire Williams', 'Maurice Williams'],
 ['Jaren Jackson Jr.', 'Jaren Jackson Jr'],
 ['Gary Trent Jr.', 'Gary Trent Jr'],
 ['P.J. Washington', 'PJ Washington'],
 ['Kelly Oubre Jr.', 'Kelly Oubre'],
 ['Troy Brown Jr.', 'Troy Brown Jr'],
 ['Isaiah Stewart', 'Isaiah Stewart II'],
 ['Dennis Smith Jr.', 'Dennis Smith Jr'],
 ['Kevin Knox II', 'Kevin Knox'],
 ['Xavier Tillman', 'Xavier Tillman Sr'],
 ['Herbert Jones', 'Perry Jones'],
 ['Kevin Porter Jr.', 'Kevin Porter Jr'],
 ['Marcus Morris Sr.', 'Marcus Morris'],
 ['James Ennis III', 'James Ennis'],
 ['Robert Williams III', 'Robert Williams'],
 ['Tim Hardaway Jr.', 'Tim Hardaway Jr'],
 ['Larry Nance Jr.', 'Larry Nance Jr'],
 ['Timothe Luwawu-Cabarrot', 'Timothe LuwawuCabarrot

From manual checking, the following matches are NOT the same person:

In [201]:
non_matches = [
    ['Brandon Williams', 'Brandon Wallace'],
    ['Ziaire Williams', 'Maurice Williams'],
    ['Herbert Jones', 'Perry Jones'],
    ['Anthony Lamb', 'Anthony Barber'],
    ['Mo Williams', 'Monty Williams'],
    ['Lazeric Jones', 'Derrick Jones'],
    ['Micah Potter', 'Michael Porter'],
    ['T.J. Williams', 'CJ Williams'],
    ['Justin Cobbs', 'Justin Bibbs'],
    ['Derrick Alston Jr.', 'Derrick Walton'],
    ['Chris Daniels', 'Chris Mills']
]

matching_names = [match for match in matching_names if match not in non_matches]
matching_names = {match[1]: match[0] for match in matching_names}

Edit the names in `salary_df` to match the ones in `df`.

In [202]:
def transform_name(name):
    if name in matching_names:
        return matching_names[name]
    return name

salary_df['PLAYER_NAME'] = salary_df['PLAYER_NAME'].apply(transform_name)

/var/folders/2w/l1f3dnqs5jg6j1jpdlt6ycvr0000gn/T/ipykernel_97934/1652976546.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df['PLAYER_NAME'] = salary_df['PLAYER_NAME'].apply(transform_name)


Re-run the merge:

In [203]:
df = df.merge(salary_df, on=['PLAYER_NAME', 'SEASON_START'], validate='many_to_one')
df

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_EST,SEASON_START,WON_GAME,SALARY,INFLATION_ADJ_SALARY
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.000,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1,10245480.0,10245480
1,1630162,Anthony Edwards,F,34:27,9.0,19.0,0.474,4.0,11.0,0.364,3.0,3.0,1.000,0.0,3.0,3.0,5.0,1.0,0.0,0.0,3.0,25.0,-3.0,MIN,Minnesota Timberwolves,2022-03-11,2021,0,10245480.0,10245480
2,1630162,Anthony Edwards,F,25:29,7.0,15.0,0.467,2.0,8.0,0.250,0.0,0.0,0.000,0.0,1.0,1.0,3.0,2.0,1.0,4.0,2.0,16.0,7.0,MIN,Minnesota Timberwolves,2022-03-09,2021,1,10245480.0,10245480
3,1630162,Anthony Edwards,F,32:22,7.0,13.0,0.538,1.0,5.0,0.200,2.0,2.0,1.000,2.0,2.0,4.0,4.0,4.0,0.0,4.0,5.0,17.0,7.0,MIN,Minnesota Timberwolves,2022-02-28,2021,1,10245480.0,10245480
4,1630162,Anthony Edwards,F,37:46,5.0,13.0,0.385,1.0,6.0,0.167,4.0,6.0,0.667,1.0,2.0,3.0,5.0,1.0,1.0,2.0,3.0,15.0,-21.0,MIN,Minnesota Timberwolves,2022-02-25,2021,0,10245480.0,10245480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492307,1628473,Yakuba Ouattara,NaN,3:09,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,-7.0,BKN,Brooklyn Nets,2017-10-03,2017,1,77250.0,85683
492308,201582,Alexis Ajinca,NaN,8:30,0.0,1.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,-10.0,NOP,New Orleans Pelicans,2017-10-13,2017,0,4961798.0,5503462
492309,201582,Alexis Ajinca,NaN,11:49,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,0.000,0.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,-1.0,NOP,New Orleans Pelicans,2017-10-06,2017,0,4961798.0,5503462
492310,1627818,Kaleb Tarczewski,NaN,20:55,2.0,3.0,0.667,0.0,0.0,0.000,1.0,2.0,0.500,1.0,9.0,10.0,0.0,0.0,2.0,2.0,3.0,5.0,-12.0,OKC,Oklahoma City Thunder,2016-10-13,2016,0,75000.0,84546


### Add Team Payroll to DataFrame

Before saving the dataset, we still need to account for the growth of the NBA, the payroll of each team, and the effects of inflation. To do this, we get team payroll data from https://hoopshype.com/salaries/.

In [204]:
# total_payroll_df = pd.DataFrame(columns=['Team', 'SEASON_START', 'TEAM_PAYROLL', 'INFLATION_ADJ_TEAM_PAYROLL'])

# for season in range(2003, 2022):
#     if season == 2021:
#         payroll_df = pd.read_html(f'https://hoopshype.com/salaries/')[0]
#     elif season == 2006:
#         # currently unable to be queried on hoopshype.com
#         payroll_df = pd.read_html('https://web.archive.org/web/20220324105246/https://hoopshype.com/salaries/2006-2007/')[1]
#     else:
#         payroll_df = pd.read_html(f'https://hoopshype.com/salaries/{season}-{season+1}/')[0]
#     season_str = f'{season}/{str(season+1)[-2:]}'
#     payroll_df = payroll_df.rename(columns={season_str: 'TEAM_PAYROLL', season_str + '(*)': 'INFLATION_ADJ_TEAM_PAYROLL'})
#     if season == 2021:
#         payroll_df['INFLATION_ADJ_TEAM_PAYROLL'] = payroll_df['TEAM_PAYROLL'] # no inflation
#     payroll_df = payroll_df[['Team', 'TEAM_PAYROLL', 'INFLATION_ADJ_TEAM_PAYROLL']]
#     payroll_df['SEASON_START'] = season
#     total_payroll_df = pd.concat([total_payroll_df, payroll_df])

# # Save as raw CSV in the /data folder
# total_payroll_df.to_csv('../data/raw/csv/payroll.csv', index=False)

# Given that we have already run the code above
total_payroll_df = pd.read_csv('../data/raw/csv/payroll.csv')
total_payroll_df

Map the team city names to the actual name of the team.

In [145]:
teams_dict = {
    'Golden State': 'Golden State Warriors',
    'Brooklyn': 'Brooklyn Nets',
    'LA Clippers': 'Los Angeles Clippers',
    'LA Lakers': 'Los Angeles Lakers',
    'Milwaukee': 'Milwaukee Bucks',
    'Utah': 'Utah Jazz',
    'Philadelphia': 'Philadelphia 76ers',
    'Miami': 'Miami Heat',
    'Denver': 'Denver Nuggets',
    'Indiana': 'Indiana Pacers',
    'Boston': 'Boston Celtics',
    'Minnesota': 'Minnesota Timberwolves',
    'Phoenix': 'Phoenix Suns',
    'Cleveland': 'Cleveland Cavaliers',
    'Atlanta': 'Atlanta Hawks',
    'Chicago': 'Chicago Bulls',
    'New Orleans': 'New Orleans Pelicans',
    'Toronto': 'Toronto Raptors',
    'Houston': 'Houston Rockets',
    'Detroit': 'Detroit Pistons',
    'Sacramento': 'Sacramento Kings',
    'Washington': 'Washington Wizards',
    'San Antonio': 'San Antonio Spurs',
    'Orlando': 'Orlando Magic',
    'Portland': 'Portland Trail Blazers',
    'Dallas': 'Dallas Mavericks',
    'New York': 'New York Knicks',
    'Charlotte': 'Charlotte Hornets',
    'Memphis': 'Memphis Grizzlies',
    'Oklahoma City': 'Oklahoma City Thunder'
}

total_payroll_df['TEAM_NAME'] = total_payroll_df['Team'].apply(lambda name: teams_dict[name])
total_payroll_df = total_payroll_df.drop(columns=['Team'])
total_payroll_df['TEAM_PAYROLL'] = pd.to_numeric(total_payroll_df['TEAM_PAYROLL'].str.replace('[$,]', ''))
total_payroll_df['INFLATION_ADJ_TEAM_PAYROLL'] = pd.to_numeric(total_payroll_df['INFLATION_ADJ_TEAM_PAYROLL'].str.replace('[$,]', ''))

total_payroll_df

/var/folders/2w/l1f3dnqs5jg6j1jpdlt6ycvr0000gn/T/ipykernel_97934/4055151519.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  total_payroll_df['INFLATION_ADJ_TEAM_PAYROLL'] = pd.to_numeric(total_payroll_df['INFLATION_ADJ_TEAM_PAYROLL'].str.replace('[$,]', ''))


,SEASON_START,TEAM_PAYROLL,INFLATION_ADJ_TEAM_PAYROLL,TEAM_NAME
0,2003,89444820,132290680,New York Knicks
1,2003,82956768,122694724,Portland Trail Blazers
2,2003,79536723,117636409,Dallas Mavericks
3,2003,71272042,105412780,Minnesota Timberwolves
4,2003,68372826,101124779,Sacramento Kings
...,...,...,...,...
25,2021,124600538,124600538,Dallas Mavericks
26,2021,122624081,122624081,New York Knicks
27,2021,122139566,122139566,Charlotte Hornets
28,2021,117284457,117284457,Memphis Grizzlies


Join the original dataframe with the team payroll dataframe.

In [146]:
df = df.merge(total_payroll_df, on=['TEAM_NAME', 'SEASON_START'])
df

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,PLUS_MINUS,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_EST,SEASON_START,WON_GAME,SALARY,INFLATION_ADJ_SALARY,TEAM_PAYROLL,INFLATION_ADJ_TEAM_PAYROLL
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,...,5.0,MIN,Minnesota Timberwolves,2022-03-12,2021,1,10245480.0,10245480,137098327,137098327
1,1630162,Anthony Edwards,F,34:27,9.0,19.0,0.474,4.0,11.0,0.364,...,-3.0,MIN,Minnesota Timberwolves,2022-03-11,2021,0,10245480.0,10245480,137098327,137098327
2,1630162,Anthony Edwards,F,25:29,7.0,15.0,0.467,2.0,8.0,0.250,...,7.0,MIN,Minnesota Timberwolves,2022-03-09,2021,1,10245480.0,10245480,137098327,137098327
3,1630162,Anthony Edwards,F,32:22,7.0,13.0,0.538,1.0,5.0,0.200,...,7.0,MIN,Minnesota Timberwolves,2022-02-28,2021,1,10245480.0,10245480,137098327,137098327
4,1630162,Anthony Edwards,F,37:46,5.0,13.0,0.385,1.0,6.0,0.167,...,-21.0,MIN,Minnesota Timberwolves,2022-02-25,2021,0,10245480.0,10245480,137098327,137098327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492307,1629649,Ignas Brazdeikis,NaN,4:59,1.0,2.0,0.500,0.0,1.0,0.000,...,-1.0,NYK,New York Knicks,2020-01-08,2019,0,898310.0,952855,100232129,106318213
492308,1629649,Ignas Brazdeikis,NaN,6:40,0.0,2.0,0.000,0.0,2.0,0.000,...,-6.0,NYK,New York Knicks,2019-10-11,2019,0,898310.0,952855,100232129,106318213
492309,1629649,Ignas Brazdeikis,NaN,5:25,0.0,1.0,0.000,0.0,1.0,0.000,...,-2.0,NYK,New York Knicks,2019-12-10,2019,0,898310.0,952855,100232129,106318213
492310,1629649,Ignas Brazdeikis,NaN,3:43,0.0,2.0,0.000,0.0,0.0,0.000,...,4.0,NYK,New York Knicks,2019-11-06,2019,0,898310.0,952855,100232129,106318213


Check that all rows have a `TEAM_PAYROLL` and `INFLATION_ADJ_TEAM_PAYROLL` value:

In [147]:
df['TEAM_PAYROLL'].isnull().sum(), df['INFLATION_ADJ_TEAM_PAYROLL'].isnull().sum()

(0, 0)

Calculate the total NBA payroll for each year:

In [148]:
league_payroll_df = total_payroll_df.groupby('SEASON_START').sum().rename(columns={
    'TEAM_PAYROLL': 'LEAGUE_PAYROLL', 'INFLATION_ADJ_TEAM_PAYROLL': 'INFLATION_ADJ_LEAGUE_PAYROLL'
})

league_payroll_df

,LEAGUE_PAYROLL,INFLATION_ADJ_LEAGUE_PAYROLL
SEASON_START,,
2003,1672617482,2473834730
2004,1773666037,2540315861
2005,1897469981,2650565322
2006,1939373647,2596944366
2007,2063887234,2691358145
2008,2158512738,2680160073
2009,2113114155,2661767475
2010,2025870243,2525271450
2011,2016175240,2426820122


In [149]:
df = df.merge(league_payroll_df, on='SEASON_START')
df

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,TEAM_NAME,GAME_DATE_EST,SEASON_START,WON_GAME,SALARY,INFLATION_ADJ_SALARY,TEAM_PAYROLL,INFLATION_ADJ_TEAM_PAYROLL,LEAGUE_PAYROLL,INFLATION_ADJ_LEAGUE_PAYROLL
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,...,Minnesota Timberwolves,2022-03-12,2021,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242
1,1630162,Anthony Edwards,F,34:27,9.0,19.0,0.474,4.0,11.0,0.364,...,Minnesota Timberwolves,2022-03-11,2021,0,10245480.0,10245480,137098327,137098327,4125163242,4125163242
2,1630162,Anthony Edwards,F,25:29,7.0,15.0,0.467,2.0,8.0,0.250,...,Minnesota Timberwolves,2022-03-09,2021,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242
3,1630162,Anthony Edwards,F,32:22,7.0,13.0,0.538,1.0,5.0,0.200,...,Minnesota Timberwolves,2022-02-28,2021,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242
4,1630162,Anthony Edwards,F,37:46,5.0,13.0,0.385,1.0,6.0,0.167,...,Minnesota Timberwolves,2022-02-25,2021,0,10245480.0,10245480,137098327,137098327,4125163242,4125163242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492307,203129,Tornike Shengelia,NaN,4:59,0.0,1.0,0.000,0.0,1.0,0.000,...,Brooklyn Nets,2013-01-08,2012,1,473604.0,560734,87644649,103768982,2015938307,2386818417
492308,203129,Tornike Shengelia,NaN,1:36,0.0,0.0,0.000,0.0,0.0,0.000,...,Brooklyn Nets,2013-01-02,2012,1,473604.0,560734,87644649,103768982,2015938307,2386818417
492309,203129,Tornike Shengelia,NaN,3:20,0.0,0.0,0.000,0.0,0.0,0.000,...,Brooklyn Nets,2012-12-23,2012,1,473604.0,560734,87644649,103768982,2015938307,2386818417
492310,202345,Damion James,NaN,0:17,0.0,0.0,0.000,0.0,0.0,0.000,...,Brooklyn Nets,2013-01-15,2012,1,50258.0,59504,87644649,103768982,2015938307,2386818417


### Calculate additional columns based off payroll

Based off the payroll of each team and the entire league, we can add additional columns:

1. The fraction of team payroll that a player's salary takes up (`TEAM_IMPORTANCE`)
2. The fraction of the league's payroll that a player's salary takes up (`LEAGUE_IMPORTANCE`)
3. The fraction of the league's payroll that a team's payroll takes up (`TEAM_MARKET_SIZE`)

These fractions are inflation-independent (would simply be multiplying the same inflation factor to the numerator and denominator).

In [152]:
df['TEAM_IMPORTANCE'] = df['SALARY'] / df['TEAM_PAYROLL']
df['LEAGUE_IMPORTANCE'] = df['SALARY'] / df['LEAGUE_PAYROLL']
df['TEAM_MARKET_SIZE'] = df['TEAM_PAYROLL'] / df['LEAGUE_PAYROLL']
df

,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,WON_GAME,SALARY,INFLATION_ADJ_SALARY,TEAM_PAYROLL,INFLATION_ADJ_TEAM_PAYROLL,LEAGUE_PAYROLL,INFLATION_ADJ_LEAGUE_PAYROLL,TEAM_IMPORTANCE,LEAGUE_IMPORTANCE,TEAM_MARKET_SIZE
0,1630162,Anthony Edwards,F,36:22,4.0,10.0,0.400,3.0,8.0,0.375,...,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242,0.074731,0.002484,0.033235
1,1630162,Anthony Edwards,F,34:27,9.0,19.0,0.474,4.0,11.0,0.364,...,0,10245480.0,10245480,137098327,137098327,4125163242,4125163242,0.074731,0.002484,0.033235
2,1630162,Anthony Edwards,F,25:29,7.0,15.0,0.467,2.0,8.0,0.250,...,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242,0.074731,0.002484,0.033235
3,1630162,Anthony Edwards,F,32:22,7.0,13.0,0.538,1.0,5.0,0.200,...,1,10245480.0,10245480,137098327,137098327,4125163242,4125163242,0.074731,0.002484,0.033235
4,1630162,Anthony Edwards,F,37:46,5.0,13.0,0.385,1.0,6.0,0.167,...,0,10245480.0,10245480,137098327,137098327,4125163242,4125163242,0.074731,0.002484,0.033235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492307,203129,Tornike Shengelia,NaN,4:59,0.0,1.0,0.000,0.0,1.0,0.000,...,1,473604.0,560734,87644649,103768982,2015938307,2386818417,0.005404,0.000235,0.043476
492308,203129,Tornike Shengelia,NaN,1:36,0.0,0.0,0.000,0.0,0.0,0.000,...,1,473604.0,560734,87644649,103768982,2015938307,2386818417,0.005404,0.000235,0.043476
492309,203129,Tornike Shengelia,NaN,3:20,0.0,0.0,0.000,0.0,0.0,0.000,...,1,473604.0,560734,87644649,103768982,2015938307,2386818417,0.005404,0.000235,0.043476
492310,202345,Damion James,NaN,0:17,0.0,0.0,0.000,0.0,0.0,0.000,...,1,50258.0,59504,87644649,103768982,2015938307,2386818417,0.000573,0.000025,0.043476


##

### Save the final dataframe.

In [156]:
df.to_csv('../data/clean/stats_salaries.csv', index=False)